# pytorchで簡単にニューラルネットワークを作ろう

## ニューラルネットワークとは

- できることを見ると
  - 回帰問題（数値予測）と分類問題が解けるやつ
  - 入力と期待される出力の組（教師データ）を与えればいい感じに補間してくれるやつ
    - 損失関数は上手に設定する必要がある
- 構成するパーツを見ると
  - 人工ニューロンがいっぱい繋がったやつ

## 人工ニューロンとは（数式は覚えなくていい）

- $d$ 個の入力（=入力ベクトル）$\boldsymbol{x}=(1,x_1,\dots,x_d)^{T}$ を受け、
  重み付け和を計算し、適当な関数（活性化関数）に通して出力するやつ
  - 単に内積で表せるようにダミーの $1$ を第0次元に付け足している（cf. 同次座標系）
- ニューロンの持つパラメータは
  - 重み $\boldsymbol{w}=(w_0,w_1,\dots,w_d)^{T}$
  - 活性化関数 $f$
    - しばしば同じ層のニューロンは同じ活性化関数を使う
- 出力は $f(\boldsymbol{w}^{T}\boldsymbol{x})=f(w_0+w_1x_1+\dots+w_dx_d)$

![ニューラルネットワークは多数のニューロンが接続されている](./resources/figures/neural_network.png)
![ニューロンの入出力](./resources/figures/artificial_neuron.png)

## ニューラルネットワークの万能近似定理（普遍性定理）

3層ニューラルネットワークはいくつかの条件のもと、その重みを調節することによって出力の誤差を任意の正数まで減らすことができる

現在は活性化関数はSigmoidalな関数でなくても良いとされている

## ニューラルネットワークで分類問題を解くには

$c$ クラス分類問題を解くならば、$c$ 出力のニューラルネットワークにする

入力を与えてForward計算をしたとき、出力層において最大の値を取っているノードのクラスに分類されたとする

## 画像分類問題を解く

### CIFAR-10とは

32x32サイズのRGBカラー画像が、10クラス各6000個用意されたデータセット。

### ニューラルネットワークに画像を入力する（一般論として）

サイズ $A \times B$、3チャンネル（RGB）固定であるとするならば、flattenして $3AB$ 次元のベクトルとして入力すれば良い

### CIFAR-10の場合

32x32 のRGB画像なので、入力は32x32x3=3072次元とする

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np

# 全結合 3-Layer FFN の定義
class FFN(torch.nn.Module):
    def __init__(self, input: int, hidden: int, output: int) -> None:
        # モデルの重みの定義
        super().__init__()
        self.flatten = nn.Flatten()
        self.__fc1 = nn.Linear(input, hidden)
        self.__fc2 = nn.Linear(hidden, output)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # 入力から出力を計算する
        x = self.flatten(x)
        x = self.__fc1(x)
        x = F.relu(x)
        x = self.__fc2(x)
        return x


def train_loop(model, dataloader, loss_fn, optimizer) -> None:
    # 学習用関数の定義
    num_classes = 10
    size = len(dataloader.dataset)
    for batch, (x, y) in enumerate(dataloader):
        pred = model(x)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

        
def test_loop(model, dataloader, loss_fn) -> None:
    # 評価用関数の定義
    num_classes = 10
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    confusion_matrix = np.zeros((num_classes, num_classes), dtype=int)
    
    with torch.no_grad():
        for x, y in dataloader:
            pred = model(x)
            loss = loss_fn(pred, y)
            test_loss += loss.item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            confusion_matrix[pred.argmax(1).detach().numpy(), y] += 1
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    print(confusion_matrix)
        

if __name__ == '__main__':
    # CIFAR-10 データセットを読み込む
    cifar10_data_train = torchvision.datasets.CIFAR10('./', train=True,  download=True, transform=torchvision.transforms.ToTensor())
    cifar10_data_test  = torchvision.datasets.CIFAR10('./', train=False, download=True, transform=torchvision.transforms.ToTensor())
    # DataLoader の設定
    dataloader_train = torch.utils.data.DataLoader(cifar10_data_train, batch_size=50, shuffle=True)
    dataloader_test  = torch.utils.data.DataLoader(cifar10_data_test,  batch_size=50, shuffle=True)
    
    ffn = FFN(input=3072, hidden=1000, output=10)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(ffn.parameters())
    
    epochs = 5
    for i in range(epochs):
        print(f"Epoch {i+1}\n-------------------------------")
        train_loop(ffn, dataloader_train, loss_fn, optimizer)
        test_loop(ffn, dataloader_test, loss_fn)

Files already downloaded and verified
Files already downloaded and verified
Epoch 1
-------------------------------
loss: 2.299946  [    0/50000]
loss: 1.948577  [ 5000/50000]
loss: 2.126779  [10000/50000]
loss: 1.857857  [15000/50000]
loss: 1.899544  [20000/50000]
loss: 1.654425  [25000/50000]
loss: 1.673904  [30000/50000]
loss: 2.124661  [35000/50000]
loss: 1.670381  [40000/50000]
loss: 1.508015  [45000/50000]
Test Error: 
 Accuracy: 38.6%, Avg loss: 1.703821 

[[179  33  82  27  43  20   4  34  97  43]
 [ 71 192  31  81  35  40  53  56 117 167]
 [ 91  20 158  94 134 109 106  74  23  12]
 [ 18  23  44 118  31 101  45  34  17  21]
 [ 63  43 147 119 179 112 135 142  41  41]
 [  4  18  45 103  23 140  44  33  24  15]
 [ 20  25  70  76  73  68 175  42   3  31]
 [ 28  23  42  44  40  49  13 169  23  31]
 [127  74  35  29  27  39  21  31 188  91]
 [ 26  62  11  31  10  15  13  57  42 156]]
Epoch 2
-------------------------------
loss: 1.937411  [    0/50000]
loss: 1.719023  [ 5000/50000]
l